# Summary

Here we use the modular_alignment_2d functions to assign alignments to galaxies with 2D information.</br>
In these cases, we either don't have the full 3D information available, or we want to use inherently 2D</br>
properties like the tidal tensor.

In [9]:
from modular_alignment_2d import align_to_axis, align_to_tidal_field, align_radially, tidal_angle
from astropy.io import ascii
import numpy as np

In [6]:
# This dict is just a mapping between what I call the value and the key used in the astropy table
# Maybe a slight bit hacky, but it lets me use a consistent naming scheme as long as the user provides some similar dict

table_keys = {
    "ra" : "ra_true",                                                   # RA
    "dec" : "dec_true",                                                 # dec
    "redshift" : "redshiftHubble",                                      # Redshift
    "x" : "x",
    "y" : "y",
    "z" : "z",
    "halo_x" : "baseDC2/target_halo_x",
    "halo_y" : "baseDC2/target_halo_y",
    "halo_z" : "baseDC2/target_halo_z",
    "halo_axisA_x" : "baseDC2/target_halo_axis_A_x",
    "halo_axisA_y" : "baseDC2/target_halo_axis_A_y",
    "halo_axisA_z" : "baseDC2/target_halo_axis_A_z",
    "halo_mvir" : "baseDC2/host_halo_mvir",
    "isCentral" : "isCentral",
    "ellipticity" : "morphology/totalEllipticity",
    "e1" : "morphology/totalEllipticity1",
    "e2" : "morphology/totalEllipticity2",
    "gamma1" : "shear1",
    "gamma2" : "shear2",
    "g" : "mag_true_g",
    "r" : "mag_true_r",
    "stellar_mass" : "stellar_mass",
    "s11" : "tidal_s_11",
    "s12" : "tidal_s_12",
    "s22" : "tidal_s_22"
}

Read in a short example table.</br>
One central galaxy and two satellites (not necessarily related)

In [7]:
data = ascii.read("short_table.dat")
data["isCentral"] = data["isCentral"] == "True"

In [8]:
data

ra_true,dec_true,redshiftHubble,x,y,z,baseDC2/target_halo_x,baseDC2/target_halo_y,baseDC2/target_halo_z,baseDC2/target_halo_axis_A_x,baseDC2/target_halo_axis_A_y,baseDC2/target_halo_axis_A_z,baseDC2/host_halo_mvir,baseDC2/target_halo_mass,isCentral,morphology/totalEllipticity,morphology/totalEllipticity1,morphology/totalEllipticity2,shear1,shear2,mag_true_g,mag_true_r,stellar_mass,tidal_s_11,tidal_s_12,tidal_s_22
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
0.6554793009305763,-40.187277441739155,0.20298659801483154,445.3499755859375,5.095147132873535,-376.2050476074219,445.3499755859375,5.095147132873535,-376.2050476074219,-0.06459414886164111,-0.054491909701089906,0.12305640417054158,126449999872.0,158923784192.0,True,0.5069335103034973,-0.4976382255554199,0.0966322124004364,-0.0010780795044078029,-0.0001935919043020633,22.277347564697266,21.623077392578125,475602400.0,-0.2818889944672113,0.40005400124426527,0.38830382637543054
0.702982389058322,-39.47357707949621,0.20586907863616943,456.1093484190837,5.596449378418383,-375.6626645289206,456.0932922363281,5.665058135986328,-375.6200256347656,0.3174143488982946,-0.051942194453338246,0.2659174734755303,2824100118528.0,3472242507776.0,False,0.03980273753404617,-0.01758822798728943,0.03570591285824776,4.247972215298823e-05,-0.001404673295386511,22.53379249572754,21.499557495117188,1453857664.0,0.2174605605253552,0.36815189641080087,0.8146693157568672
0.7200102146955414,-39.482723243105056,0.20586907863616943,456.02346422424995,5.730942836655683,-375.71550924438276,456.0932922363281,5.665058135986328,-375.6200256347656,0.3174143488982946,-0.051942194453338246,0.2659174734755303,2824100118528.0,3472242507776.0,False,0.45927709341049194,0.20627683401107788,-0.4103478193283081,-2.3903166125993636e-05,-0.0009506534437888547,21.415847778320312,20.174400329589844,7387039232.0,-0.0837378781577978,0.4144596483241478,0.6740889367478052


# Align

## Get The Necessary Variables

### From the data table
Here, I am grabbing them from an existing galaxy catalog. However, as long as you get these values (regardless of how), that's all the next step needs.

In [14]:
# First, get the mask for central galaxies
central_mask = data["isCentral"]

# Second, get the values from the appropriate columns
# Here's where that weird dict above comes in handy

# Major axes
halo_axisA_x = data[ table_keys["halo_axisA_x"] ]                # Grab the column with whatever the catalog's version of "halo_axisA_x" is called
halo_axisA_y = data[ table_keys["halo_axisA_y"] ]
halo_axisA_z = data[ table_keys["halo_axisA_z"] ]

# Central/Parent halo positions
halo_x = data[ table_keys["halo_x"] ]
halo_y = data[ table_keys["halo_y"] ]
halo_z = data[ table_keys["halo_z"] ]

# Galaxy positions
x = data[ table_keys["x"] ]
y = data[ table_keys["y"] ]
z = data[ table_keys["z"] ]

redshift = data[ table_keys["redshift"] ]

s11 = data[ table_keys["s11"] ]
s12 = data[ table_keys["s12"] ]
s22 = data[ table_keys["s22"] ]

### Extra variables
We also need alignment strengths and the box size

In [11]:
central_alignment_strength = 0.8
satellite_alignment_strength=0.5

Lbox = np.array([500., 500., 500.])

## The Actual Alignment
Perform the alignment and get the galaxy major axis orientation.</br>
We also get the intermediate and minor axes, but these are just random perpendicular vectors

In [16]:
tidal_field_angles = tidal_angle(s11, s22, s12, redshift)
print(tidal_field_angles)

    tidal_s_12    
------------------
 2.704828216715141
2.6969278615010217
 2.726506572405553


In [17]:
angs = align_to_tidal_field(s11, s22, s12, redshift, 1.)
print(angs)

[2.7199358  2.82840556 2.74813541]


/media/nvanalfen/T7/Northeastern/Research/Coherent_2D_3D/modular_alignments/vonmises_distribution.py:117: UserWarning: The maximum number of iterations reached, random variates may not be representative.
  warn(msg)
